# C3 - diagnostics Extract
#### based on - C3.ai COVID-19 Data Lake Quickstart in Python
#### Location Exposure doco: https://c3.ai/covid-19-api-documentation/#tag/Diagnosis/paths/~1api~11~1diagnosis~1fetch/post

Loading Dependencies Below:

In [45]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from scipy.stats import gamma
import numpy as np

Ensuring that the most recent version of pandas (>= 1.0.0)

In [46]:
print("pandas version", pd.__version__)
assert pd.__version__[0] >= "1", "To use this notebook, upgrade to the newest version of pandas. See https://pandas.pydata.org/pandas-docs/stable/getting_started/install.html for details."

pandas version 1.1.3


<a id="helpers"></a>
## Helper methods for accessing the API

The helper methods in `c3aidatalake.py` convert a JSON response from the C3.ai APIs to a Pandas DataFrame. You may wish to view the code in `c3aidatalake.py` before running the code below. 

`c3aidatalake.py` is a API file provided by C3.ai

In [47]:
import c3aidatalake 

In [48]:
def read_data(url, body, c3_source):
    has_more = True
    body['spec']['offset'] = 0
    df = pd.DataFrame()
    while has_more:
        print(body['spec']['offset'])
        response = requests.post(
            url,
            json = body,
            headers = {'Accept' : 'application/json'}
        )
        new_df = pd.json_normalize(response.json()['objs'])
        df = df.append(new_df)
        has_more = response.json()['hasMore']
        body['spec']['offset'] += new_df.shape[0]
    # return df
    diagnoses = df

    diagnoses_long = diagnoses.explode("diagnostics")
    diagnoses_long = pd.concat([
        diagnoses_long.reset_index(),
        pd.json_normalize(
            diagnoses_long.loc[diagnoses_long.source != 'UCSD', "diagnostics"]
        )[["key", "value"]]
    ], axis = 1).drop(columns = "diagnostics")
    diagnoses_long.to_csv(datadir + 'diagnoses_long_' + c3_source + '.csv')
    diagnoses_long

    diagnoses_wide = (
        diagnoses_long
        .loc[~diagnoses_long[['key', 'value']].isna().all(axis=1)]
        .pivot(columns = "key", values = "value")
    )
    diagnoses_wide = pd.concat([diagnoses, diagnoses_wide], axis = 1).drop(columns = "diagnostics")
    diagnoses_wide.to_csv(datadir + 'diagnoses_wide_' + c3_source + '.csv')
    diagnoses_wide

In [49]:
datadir = 'C:/Dev/Smart-Cookies/C3ai/'

url = 'https://api.c3.ai/covid/api/1/diagnosis/fetch'

body = {
      "spec" : {
          "filter" : "contains(source, 'Braid')", 
          "include" : "this, diagnostics.source, diagnostics.key, diagnostics.value"
      }
  }

diagnosis = read_data(url, body, 'Braid')
diagnosis

body = {
      "spec" : {
          "filter" : "contains(source, 'Montreal')", 
          "include" : "this, diagnostics.source, diagnostics.key, diagnostics.value"
      }
  }

diagnosis = read_data(url, body, 'Montreal')
diagnosis

0
0


In [50]:
# diagnoses = c3aidatalake.fetch(
#   "diagnosis",
#   {
#       "spec" : {
#           "filter" : "contains(testResults, 'COVID-19')", 
#           "include" : "this, diagnostics.source, diagnostics.key, diagnostics.value"
#       }
#   }
# )

# diagnoses